<a href="https://colab.research.google.com/github/raviteja1432/NNDL/blob/main/ASSIGNMENT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/Share 'Active Power Load - 33_11KV - Godishala Substation  (2).xlsx'")
data.head()

,DATE,TIME,VOLTAGE,CURRENT,PF,POWER (KW),"""WEEKEND/WEEKDAY""",SEASON,Temp (F),Humidity (%),Substation Shutdown,F1,F2,F3,F4,Jul-Oct-,Rainy,0.0
0,2021-01-01,01-00,11.6,102.0,0.96,1967.388015,0.0,1.0,65.0,90.0,NaN,NaN,NaN,NaN,NaN,Nov-Feb,Winter,1.0
1,NaT,02-00,11.6,102.0,0.96,1967.388015,0.0,1.0,65.0,90.0,NaN,NaN,NaN,NaN,NaN,Mar-Jun,Summer,2.0
2,NaT,03-00,11.6,102.0,0.96,1967.388015,0.0,1.0,65.0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaT,04-00,11.3,130.0,0.96,2442.607331,0.0,1.0,78.0,49.0,NaN,NaN,NaN,NaN,NaN,Weekend,1.0,NaN
4,NaT,05-00,11.2,148.0,0.96,2756.205522,0.0,1.0,78.0,49.0,NaN,NaN,NaN,NaN,NaN,Weekday,0.0,NaN


In [ ]:
#forming  the dataset 
x = []
y = []
for i in range(23, data.shape[0]):
    x.append(data['POWER (KW)'].iloc[i-23]) #Since x is previous day one hour before of y
    y.append(data['POWER (KW)'].iloc[i])
data = pd.DataFrame({'previous day one hour before Load in kw' : x, 'Present Hour Load in kw' : y})
data.head()

,previous day one hour before Load in kw,Present Hour Load in kw
0,1967.388015,412.034103
1,1967.388015,1828.915749
2,1967.388015,1828.915749
3,2442.607331,1828.915749
4,2756.205522,1892.265507


In [ ]:
#normalizing
maxX = np.max(data['previous day one hour before Load in kw'])
minX = np.min(data['previous day one hour before Load in kw'])
maxY = np.max(data['Present Hour Load in kw'])
minY = np.min(data['Present Hour Load in kw'])

data['previous day one hour before Load in kw'] = (data['previous day one hour before Load in kw'] - minX) / (maxX - minX)
data['Present Hour Load in kw'] = (data['Present Hour Load in kw']- minY) / (maxY - minY)

In [ ]:
from sklearn.model_selection import train_test_split
x = data['previous day one hour before Load in kw']
y = data['Present Hour Load in kw']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [ ]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 200
em2 = 0
ec2 = 0
v = 0.5
E=1E-6

# Loop for epochs
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
        # Calculating Grad M
        gradM = -1 * (trainY.iloc[i] - m* trainX.iloc[i] - c) * trainX.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (trainY.iloc[i] - m* trainX.iloc[i] - c)
        
        # Calculating updated values of vm and vc
        em2=v*(em2**2)+(1-v)*(gradM**2)
        ec2=v*(ec2**2)+(1-v)*(gradC**2)
        # Updating m and c values
        m -= (eta*gradM)/((em2+E)**0.5)
        c -=  (eta*gradC)/((ec2+E)**0.5)

# Print model parameters after training
print(f'm = {m}\nc = {c}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in double_scalars


m = -5.095155083493154
c = 1.849582181917596


In [ ]:

train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

# Denormalization of Output Values
train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
rmse = math.sqrt(mean_squared_error(train_pred, trainY)) 
print("testing error")
print('Root mean square error', rmse) 
mse = (mean_squared_error(train_pred,trainY)) 
print('Mean square error', mse) 
mae=mean_absolute_error(train_pred,trainY)
print('Mean absolute error', mae)


#Testing Accuracies
rmse = math.sqrt(mean_squared_error(test_pred,testY ))
print("testing error") 
print('Root mean square error', rmse) 
mse = (mean_squared_error(test_pred,testY))
print('Mean square error', mse) 
mae=mean_absolute_error(test_pred,testY)
print('Mean absolute error', mae)

testing error
Root mean square error 7777.526559930257
Mean square error 60489919.39042057
Mean absolute error 6599.243252207111
testing error
Root mean square error 7758.806712378947
Mean square error 60199081.600056596
Mean absolute error 6494.474266615221


In [ ]:

#checking the difference between actal value and predicted value of training data
pd.DataFrame({'Actual value' : trainY, 'Predicted value' : train_pred})

,Actual value,Predicted value
0,1137.998950,7675.285561
1,960.560737,8498.311354
2,3216.132908,-3911.145530
3,1799.593861,4175.228499
4,2897.817996,-1943.296684
...,...,...
7858,713.701928,10014.175544
7859,1484.332901,2631.053492
7860,2321.710184,2934.071009
7861,4482.478208,-8304.352371


In [ ]:
pd.DataFrame({'Actual' : testY, 'Predicted':test_pred})

,Actual,Predicted
0,981.838981,9507.095992
1,1665.193646,1685.182760
2,1195.115057,9184.230898
3,743.959123,9199.586516
4,1653.623547,3582.501667
...,...,...
869,971.427624,8859.697867
870,1011.077731,8665.616983
871,919.680874,9069.663872
872,878.814867,8322.780761


In [ ]:

# Real Time Prediction
x = float(input('Enter the power load at one hour and one day before : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted power load at present hour :', prediction)

Enter the power load at one hour and one day before : 9507
Predicted power load at present hour : -35026.47144589282
